# 6.6 Sourcing and Analyzing Time-Series Data

### Contents:

#### 1. Importing your libraries and data
#### 2. Subsetting, wrangling, and cleaning time-series data
#### 3. Time series analysis: decomposition
#### 4. Testing for stationarity


## 1. Import Libraries and Data

In [1]:
import quandl
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import statsmodels.api as sm # Using .api imports the public access version of statsmodels, which is a library that handles 
# statistical models.
import os
import warnings # This is a library that handles warnings.

warnings.filterwarnings("ignore") # Disable deprecation warnings that could indicate, for instance, a suspended library or 
# feature. These are more relevant to developers and very seldom to analysts.

plt.style.use('fivethirtyeight') # This is a styling option for how your plots will appear. More examples here:
# https://matplotlib.org/3.2.1/tutorials/introductory/customizing.html
# https://matplotlib.org/3.1.0/gallery/style_sheets/fivethirtyeight.html


In [2]:
#Define path
path = r'C:\Users\kaydo\OneDrive\Desktop\final'

In [3]:
# import housesales.csv

df = pd.read_csv(os.path.join(path, 'Data', 'Prepared Data', 'housesales.csv'),index_col = False)

In [4]:
df.columns

Index(['id', 'Date', 'price', 'bedrooms', 'bathrooms', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'sqft_basement', 'yr_built',
       'yr_renovated', 'zipcode', 'lat', 'long'],
      dtype='object')

In [5]:
df.head(10)

id      Date    price  bedrooms  bathrooms  sqft_lot  floors  \
0  2524049179  08/26/14  2000000         3       2.75     44867     1.0   
1  7922800400  08/27/14   951000         5       3.25     14342     2.0   
2  8096000060  04/13/15   655000         2       1.75     15798     2.0   
3  2025069065  09/29/14  2400000         4       2.50      8354     1.0   
4  2123039032  10/27/14   369900         1       0.75     10079     1.0   
5  3225069065  06/24/14  3075000         4       5.00     18641     1.0   
6  1860600135  05/02/14  2384000         5       2.50      9050     2.0   
7  1560930070  09/11/14   840000         4       3.50     40139     1.0   
8   622049114  02/18/15  2125000         3       2.50     24069     2.0   
9  2998800125  07/01/14   730000         2       2.25      4920     1.5   

   waterfront  view  condition  sqft_basement  yr_built  yr_renovated  \
0           0     4          3            720      1968             0   
1           0     4          4              0      1968             0   
2           1     4          3            220      1915          1978   
3           1     4          3           1820      2000             0   
4           1     4          5              0      1936             0   
5           1     4          3           1950      2002             0   
6           0     4          5            280      1921             0   
7           0     4          4              0      1986             0   
8           1     4          4              0      1976             0   
9           0     4          4            600      1941             0   

   zipcode      lat     long  
0    98040  47.5316 -122.233  
1    98008  47.5880 -122.116  
2    98166  47.4497 -122.375  
3    98074  47.6338 -122.072  
4    98070  47.4683 -122.438  
5    98074  47.6053 -122.077  
6    98119  47.6345 -122.367  
7    98038  47.4010 -122.026  
8    98166  47.4169 -122.348  
9    98116  47.5730 -122.409

In [6]:
type(df)

pandas.core.frame.DataFrame

## 2. Subsetting wrangling, and cleaning time-series data

In [7]:
#Create a subset of the data that includes 'Date' and Value AKA Price
#This subset will be date and price to see how time has impacted the price of estates

dc= df[['price','Date' ]]

In [8]:
dc.head(10)

price      Date
0  2000000  08/26/14
1   951000  08/27/14
2   655000  04/13/15
3  2400000  09/29/14
4   369900  10/27/14
5  3075000  06/24/14
6  2384000  05/02/14
7   840000  09/11/14
8  2125000  02/18/15
9   730000  07/01/14

In [9]:
dc.shape

(21436, 2)

In [10]:
dc.columns

Index(['price', 'Date'], dtype='object')

In [11]:
#Concert the data columns to type DATETIME
dc['Date'] = pd.to_datetime(dc['Date'])

In [12]:
dc.dtypes

price             int64
Date     datetime64[ns]
dtype: object

In [13]:
dc.columns

Index(['price', 'Date'], dtype='object')

In [14]:
#sort dataframe by date
dc= dc.loc[(dc['Date'] >= '2014-05-02') & (dc['Date'] < '2015-05-27')]

In [15]:
dc= dc.sort_values('Date')

In [16]:
# Set the "Date" column as the index

from datetime import datetime

dc['datetime'] = pd.to_datetime(dc['Date']) # Create a datetime column from "Date.""
dc = dc.set_index('datetime') # Set the datetime as the index of the dataframe.
dc.drop(['Date'], axis=1, inplace=True) # Drop the "Date" column.

In [17]:
dc.head(10)

price
datetime          
2014-05-02  260000
2014-05-02  367500
2014-05-02  499950
2014-05-02  550000
2014-05-02  805000
2014-05-02  257950
2014-05-02  555000
2014-05-02  403000
2014-05-02  588500
2014-05-02  750000

In [18]:
# Check for missing values (you shouldn't have any)

dc.isnull().sum() 

price    0
dtype: int64

In [19]:
# Check for duplicates

dups = dc.duplicated()


In [20]:
dups.sum()

17422

In [21]:
# Drop duplicated values
dc.drop_duplicates

<bound method DataFrame.drop_duplicates of              price
datetime          
2014-05-02  260000
2014-05-02  367500
2014-05-02  499950
2014-05-02  550000
2014-05-02  805000
...            ...
2015-05-14  749950
2015-05-14  359000
2015-05-14  360000
2015-05-15  450000
2015-05-24  445500

[21435 rows x 1 columns]>

In [22]:
#Correct frequency showing as none
dcf = dc.asfreq('D')

ValueError: cannot reindex on an axis with duplicate labels

In [ ]:
# Plot the new data set

plt.figure(figsize=(15,5), dpi=100)
plt.plot(dc)

### 3. Time-series analysis: decomposition

In [ ]:
# Decompose the time series using an additive model

decomposition = sm.tsa.seasonal_decompose(data_sub, model='additive')